In [3]:
import sys;sys.path.extend(['E:\\reconstruction', 'E:\\cobamp', 'E:\\reconstruction\\src', 'E:\\cobamp\\src', 'E:/reconstruction'])
import cobra
import framed
import cobamp
import pandas as pd
import numpy as np
import pickle
import scipy as sci

# for testing the algorithms
from cobamp.wrappers import COBRAModelObjectReader
from reconstruction.methods.imat import IMAT
from reconstruction.methods.gimme import GIMME
from reconstruction.methods.tINIT import tINIT
from reconstruction.methods.fastcore import FASTcore
from reconstruction.reconstruction_properties import FastcoreProperties, tINITProperties, GIMMEProperties, IMATProperties

Here we are going to import the .mat file used for testing in the MATLAB COBRA package and will serve as basis for all the analysis

In [12]:
mat = sci.io.loadmat('./tests/FastCoreTest.mat')

Since the .mat file contains all the information needed for the TROPPO package to work (the S matrix, lower and upper bounds), we are going to parse it

In [16]:
model = mat['ConsistentRecon2']
S = model['S'][0][0].toarray()
lb = model['lb'][0][0].ravel()
ub = model['ub'][0][0].ravel()

Since the algorithms need expression data to begin the pruning of the original model, we are going to import data from glial cells (complete the rest of the story here).
We are also going to make different inputs accordingly to the needs of the algorithms that are goint to be tested.

In [ ]:
original_data # import of the omics data before/after preprocessing
input_data1 # this is for the A;B;C algorithms FASTCORE
input_data2 # this is for the D;E;F algorithms tINIT
input_data3, input_data3_1, input_data3_2, input_data3_3  # this is for the CORDA algorithms

From here, we are going to test the already implement algorithms for tissue reconstruction. 

First, we are going to be testing the FASTcore algorithm

In [ ]:
core_reactions = input_data1
fastcore = FASTcore(S, lb, ub, FastcoreProperties(solver = 'CPLEX', core=list(core_reactions)))
tissue_reactions = fastcore.fastcore()

With the FASTcore algorithm, we obtained X specific reactions for the data provided, generatung a model with X reactions, Y genes and Z metabolites, capable (or not) of producing biomass.

Now we're going to test the tINIT algorithm. For this case, as an input we are going to only provide reactions scores. The other properties of the algorithm are set to default (including the solver).

In [ ]:
tinit = tINIT(S, np.array(lb), np.array(ub),
				  tINITProperties(reactions_scores=input_data2, present_metabolites=[], essential_reactions=[],
								  production_weight=0.5, allow_excretion=False, no_reverse_loops=False, solver = "GUROBI"))
reactions_to_remove_from_the_model = tinit.run_tINIT()

With the tINIT algorithm, we obtained X specific reactions for the data provided, generatung a model with X reactions, Y genes and Z metabolites, capable (or not) of producing biomass.

For the CORDA algorithm, we needed four different types of sets of reactions (input_data3, input_data3_1, input_data3_2, input_data3_3). 

In [1]:
# code for the CORDA test

Regarding the CORDA reconstructed model, it has X reactions, Y genes and Z metabolites, and also capable(or not) of producing biomass

For GIMME, we apply a threshold to the expression values originally from the expression dataset.

In [2]:
# code for the GIMME test
properties = GIMMEProperties(
	exp_vector=input_data, # check what data should be here
    obj_frac=0.8,
    objectives=[np.array([0,0,0,0,0,0,0,0,1])], # check what vector to put here
    preprocess=True,
    flux_threshold=0.8
)
algorithm = GIMME(S, lb, ub, properties)
model_GIMME = algorithm.run()

With GIMME we are capable of obtaining both a flux distribution and a reconstructed model.
For the flux distribution, ...
As for the model, it is comprised with X reactions, Y genes and Z metabolites, capable (or not) of producing biomass.

For the iMAT algorihtm, we have to provide two different thresholds (...).

In [3]:
# code for the iMAT test
properties = IMATProperties(
    exp_vector = input_data, # check what input to here
    exp_thresholds=(0.2,0.5)
)

method = IMAT(S, lb, ub, properties)
print(['R'+str(i+1) for i in method.run()])

With iMAT, we obtained a reconstructed model with X reactions, Y genes and Z metabolites, and capable(or not) of producing biomass.

The following table represents the final summary of the tested algorithms.

In [1]:
# code for the table (do this with pandas or markdown)

Since we are implementing the algoirhtms based on the code of COBRA toolbox, the next table is a comparison of speed of execution. The computer used was running Windows 10 version, with a processor i7-3630QM CPU @ 2.40 GHz (8 CPU), 16GB of RAM and Python 3.5.6.

In [2]:
# code for the table with the benchmarks

With this, we can see that the implementation in Pyhton is ...
(mostly should run faster when compared to MATLAB)